In [1]:
# Krypto-Scanner Main (v14.5) — Free-CG stabil, Chart-Cache, Hybrid-Kategorien, Cap 50–1000 Mio, Buzz-Audit

# === Bootstrap: Pakete, Drive, Pfade ===
import sys, subprocess, os
def ensure(pkg):
    try: __import__(pkg)
    except ImportError: subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])
for p in ["feedparser","xlsxwriter","requests","pandas","numpy"]: ensure(p)

try:
    from google.colab import drive
    drive.mount("/content/drive")
    MODULE_DIR = "/content/drive/MyDrive/crypto_tool/"
    EXPORT_DIR = "/content/drive/MyDrive/Colab results"
    CACHE_DIR  = "/content/drive/MyDrive/crypto_tool/cache"
except Exception:
    MODULE_DIR = os.getcwd()
    EXPORT_DIR = os.path.join(os.getcwd(), "exports")
    CACHE_DIR  = os.path.join(os.getcwd(), "cache")
os.makedirs(EXPORT_DIR, exist_ok=True)
os.makedirs(CACHE_DIR, exist_ok=True)
if MODULE_DIR not in sys.path: sys.path.append(MODULE_DIR)

# === Logging/Warnungen ===
import logging as _rootlog, warnings, numpy as _np, pandas as _pd
_rootlog.basicConfig(level=_rootlog.INFO, format='[%(asctime)s] [%(levelname)s] %(message)s')
warnings.filterwarnings("ignore", category=RuntimeWarning)
_pd.options.mode.chained_assignment = None
_np.seterr(all="ignore")

# === ENV (Free-Plan) ===
os.environ.update({
    "COINGECKO_API_KEY": "CG-iM4aTeNWTc2kR2DSLEsTXWui",
    "CG_FORCE_FREE": "1",
    "CG_SKIP_AFTER_429": "1",
    "CG_MAX_ATTEMPTS": "1",
    "CG_MIN_INTERVAL_S": "3.5",
    "CG_CATS_TIME_BUDGET_S": "120",
    "PROVIDERS_CATS_TIME_BUDGET_S": "90",
    "SKIP_CATEGORIES": os.getenv("SKIP_CATEGORIES", "0"),
    "REQUIRE_MEXC": os.getenv("REQUIRE_MEXC", "1"),
    "LIGHT_BREAKOUT_ALL": os.getenv("LIGHT_BREAKOUT_ALL", "0"),
    "ALLOW_CG_FALLBACK": os.getenv("ALLOW_CG_FALLBACK", "0"),
    "BUZZ_HALF_LIFE_H": os.getenv("BUZZ_HALF_LIFE_H", "48"),
    "BUZZ_PUBLISHER_WEIGHTS": os.getenv("BUZZ_PUBLISHER_WEIGHTS", '{"coindesk":1.0,"cointelegraph":1.0,"theblock":1.1,"decrypt":0.9}')
})
# Optional Fallback-Provider:
# os.environ["CMC_API_KEY"] = "..."
# os.environ["MESSARI_API_KEY"] = "..."
# os.environ["COINPAPRIKA_API_KEY"] = "..."

# === Imports aus Modulen ===
from modules.utils import logging, time, datetime, timezone, pd, np
from modules.data_sources import cg_markets, map_tvl, update_seen_ids, cg_market_chart
from modules.pre_universe import apply_pre_universe_filters
from modules.exchanges import apply_mexc_filter, export_mexc_seed_template
from modules.features import compute_feature_block, exclusion_mask, peg_like_mask, tag_segment
from modules.scores import score_block, compute_early_score
from modules.breakout import compute_breakout_for_ids
from modules.buzz import add_buzz_metrics_for_candidates
from modules.backtest import backtest_on_snapshot
from modules.export import write_sheet, write_meta_sheet
from modules.export_helpers import make_fulldata
from modules.category_providers import enrich_categories_hybrid
from modules.cg_cache_patch import setup_cg_chart_cache

# === CG: /coins/{id}/market_chart Cache aktivieren (24h) ===
setup_cg_chart_cache(cache_dir=os.path.join(CACHE_DIR, "cg_chart"), ttl_hours=24)

# === CG Smart-Get: /markets unverändert, andere Endpunkte sanft drosseln ===
import time as _time, requests as _requests
from modules import data_sources as ds
_ORIG_CG_GET = getattr(ds, "_cg_get", None)
def _cg_get_smart(path, params=None):
    if path.strip().lower() == "/coins/markets" and callable(_ORIG_CG_GET):
        return _ORIG_CG_GET(path, params=params)
    key = os.getenv("COINGECKO_API_KEY","").strip()
    free = os.getenv("CG_FORCE_FREE","1") == "1" or not key
    base = "https://api.coingecko.com/api/v3" if free else "https://pro-api.coingecko.com/api/v3"
    headers = {"Accept":"application/json","User-Agent":"cg-screener/1.0"}
    if not free and key:
        headers["x-cg-pro-api-key"] = key
    q = dict(params or {})
    min_interval = float(os.getenv("CG_MIN_INTERVAL_S","3.5"))
    last = getattr(ds, "_CG_LAST_CALL_TS", None)
    if last is not None:
        delta = _time.perf_counter() - last
        if delta < min_interval: _time.sleep(min_interval - delta)
    url = f"{base}{path}"
    try:
        sess = getattr(ds, "_SESSION", None)
        r = (sess or _requests).get(url, headers=headers, params=q, timeout=20)
        ds._CG_LAST_CALL_TS = _time.perf_counter()
    except Exception as ex:
        ds.logging.warning(f"[cg-smart] net err on {url}: {ex}")
        return {}
    if r.status_code == 429:
        ds.logging.warning(f"[cg-smart] 429 on {url} → skip"); return {}
    if r.status_code != 200:
        ds.logging.warning(f"[cg-smart] HTTP {r.status_code} on {url}")
        try: return r.json() if r.content else {}
        except Exception: return {}
    try: return r.json()
    except Exception: return {}
if callable(_ORIG_CG_GET):
    ds._cg_get = _cg_get_smart

# === Lokaler Fallback für /coins/markets ===
import requests
def _cg_markets_fallback(vs="usd", per_page=250, pages=1):
    rows = []
    headers = {"Accept":"application/json","User-Agent":"cg-screener/1.0"}
    base = "https://api.coingecko.com/api/v3"
    for page in range(1, int(pages)+1):
        params = {
            "vs_currency": vs, "order": "market_cap_desc",
            "per_page": int(per_page), "page": int(page),
            "sparkline": "false", "price_change_percentage": "7d,30d"
        }
        r = requests.get(f"{base}/coins/markets", headers=headers, params=params, timeout=25)
        if r.status_code != 200:
            logging.warning(f"[cg-fb] HTTP {r.status_code} /coins/markets page={page}"); continue
        try: data = r.json()
        except Exception: data = []
        if isinstance(data, list): rows.extend(data)
    if not rows: return pd.DataFrame()
    df = pd.json_normalize(rows, sep="_")
    want = ["id","symbol","name","market_cap","total_volume",
            "price_change_percentage_7d_in_currency","price_change_percentage_30d_in_currency",
            "ath_change_percentage","circulating_supply"]
    return df[[c for c in want if c in df.columns]].copy()

# === Konfiguration ===
RUN_MODE = os.environ.get("RUN_MODE", "standard").strip().lower()
CFG = {
    "fast":     {"PAGES":1, "BREAKOUT_PER_SEGMENT":{"Hidden Gem":30,"Emerging":20,"Comeback":15,"Momentum Gem":15,"Balanced":20}, "BUZZ_TOPN":120, "DAYS":180, "USE_CP":False, "NEW_LISTINGS":True,  "BACKTEST":False},
    "standard": {"PAGES":4, "BREAKOUT_PER_SEGMENT":{"Hidden Gem":60,"Emerging":50,"Comeback":40,"Momentum Gem":40,"Balanced":40}, "BUZZ_TOPN":200, "DAYS":365, "USE_CP":True,  "NEW_LISTINGS":False, "BACKTEST":True}
}[RUN_MODE]

VS = os.environ.get("VS", "usd")
MIN_VOLUME_USD = float(os.environ.get("MIN_VOLUME_USD", "1000000"))
REQUIRE_MEXC = os.environ.get("REQUIRE_MEXC", "1") == "1"
SKIP_CATEGORIES = os.environ.get("SKIP_CATEGORIES", "0") == "1"

# LIGHT_BREAKOUT_ALL je Mode (env kann übersteuern)
LIGHT_BREAKOUT_ALL = True if RUN_MODE == "fast" else False
_env_lba = os.environ.get("LIGHT_BREAKOUT_ALL", None)
if _env_lba in ("0","1"):
    LIGHT_BREAKOUT_ALL = (_env_lba == "1")

USE_CRYPTOPANIC = CFG["USE_CP"]
CAP_MIN = 50_000_000
CAP_MAX = 1_000_000_000

# Zeitstempel (Berlin)
from datetime import datetime as _dt
try:
    from zoneinfo import ZoneInfo
    _TZ = ZoneInfo("Europe/Berlin")
except Exception:
    _TZ = None
_STAMP = (_dt.now(_TZ) if _TZ else _dt.now()).strftime("%Y%m%d%H%M")
EXPORT_NAME = f"Scanner_v14_5_output_{RUN_MODE}_{_STAMP}.xlsx"
EXPORT_PATH = os.path.join(EXPORT_DIR, EXPORT_NAME)

_pd.set_option("display.max_columns", 160); _pd.set_option("display.width", 220)

# === Regime-Helper ===
def _mom30_from_chart(prices):
    if not prices or len(prices) < 31: return np.nan
    p_now = float(prices[-1][1]); p_30 = float(prices[-31][1]); return (p_now/p_30 - 1.0)*100.0
def _dd_pct(prices):
    if not prices or len(prices) < 2: return np.nan
    arr = [float(p[1]) for p in prices]; p_now = arr[-1]; p_max = max(arr); return (p_now/p_max - 1.0)*100.0 if p_max>0 else np.nan
def _regime():
    try:
        btc = cg_market_chart("bitcoin", vs=VS, days=max(365, CFG["DAYS"]))
        eth = cg_market_chart("ethereum", vs=VS, days=max(365, CFG["DAYS"]))
        return {"btc_mom30": _mom30_from_chart(btc.get("prices",[])),
                "btc_dd": _dd_pct(btc.get("prices",[])),
                "eth_mom30": _mom30_from_chart(eth.get("prices",[])),
                "eth_dd": _dd_pct(eth.get("prices",[]))}
    except Exception as ex:
        logging.warning(f"[regime] {ex}")
        return {"btc_mom30": np.nan,"btc_dd": np.nan,"eth_mom30": np.nan,"eth_dd": np.nan}

# =========================
#          PIPELINE
# =========================
t_all = time.perf_counter()
logging.info(f"[0] Start {datetime.now(timezone.utc).isoformat()} | MODE={RUN_MODE} | PAGES={CFG['PAGES']}")

# [1] Universe (+Fallback)
t0 = time.perf_counter()
df = cg_markets(vs=VS, per_page=250, pages=CFG["PAGES"])
if df is None or df.empty:
    logging.warning("[main] cg_markets leer → Fallback nutzt direkten /coins/markets Abruf")
    df = _cg_markets_fallback(vs=VS, per_page=250, pages=CFG["PAGES"])
if df is None or df.empty:
    raise RuntimeError("cg_markets leer (Fallback ebenfalls leer)")
logging.info(f"[1] Märkte: {len(df)} Zeilen in {time.perf_counter()-t0:.2f}s")

# [2] Pre + Cap-Range
t0 = time.perf_counter()
df = apply_pre_universe_filters(df, min_volume_usd=MIN_VOLUME_USD)
df["market_cap"] = pd.to_numeric(df["market_cap"], errors="coerce")
df = df[(df["market_cap"] >= CAP_MIN) & (df["market_cap"] <= CAP_MAX)].copy()
logging.info(f"[2] Pre+CapRange: übrig {len(df)} in {time.perf_counter()-t0:.2f}s")

# [3] MEXC
t0 = time.perf_counter()
df = apply_mexc_filter(df, require_mexc=REQUIRE_MEXC)
try: export_mexc_seed_template(df, collisions_only=True)
except Exception: pass
if len(df) == 0 and REQUIRE_MEXC:
    raise RuntimeError("Nach MEXC-Schnitt 0 Zeilen.")
logging.info(f"[3] MEXC ok: {len(df)} in {time.perf_counter()-t0:.2f}s")

# Optional: New Listings
if CFG.get("NEW_LISTINGS", False):
    _ = update_seen_ids(df["id"].astype(str).tolist())

# [4] Kategorien (Hybrid, CG NICHT bevorzugt) + TVL
t0 = time.perf_counter()
if os.getenv("SKIP_CATEGORIES","0") != "1":
    df_tmp = df.copy()
    df_tmp["Segment"] = df_tmp.apply(tag_segment, axis=1)
    df_tmp["market_cap"] = pd.to_numeric(df_tmp["market_cap"], errors="coerce")
    df_tmp = df_tmp.sort_values("market_cap", ascending=True)
    seg_quota_for_cats = {"Hidden Gem":120, "Emerging":90, "Comeback":50, "Momentum Gem":30, "Balanced":10}
    ids_cat = []
    for seg, q in seg_quota_for_cats.items():
        ids_cat.extend(df_tmp[df_tmp["Segment"]==seg].head(q)["id"].astype(str).tolist())
    if len(ids_cat) < 300:
        extra = df_tmp[~df_tmp["id"].astype(str).isin(ids_cat)].head(300 - len(ids_cat))
        ids_cat.extend(extra["id"].astype(str).tolist())
    ids_cat = list(dict.fromkeys(ids_cat))[:300]
    cat_map = enrich_categories_hybrid(df, ids_cat, ttl_days=14, max_fetch=200, prefer_cg_first=False)
    df["Kategorie"] = df["id"].astype(str).map(cat_map).fillna("Unknown")
else:
    df["Kategorie"] = "Unknown"
df = map_tvl(df)
logging.info(f"[4] Kategorien/TVL ok in {time.perf_counter()-t0:.2f}s")

# [5] Features
t0 = time.perf_counter()
df = compute_feature_block(df)
logging.info(f"[5] Features ok in {time.perf_counter()-t0:.2f}s")

# [6] Segmente
df["Segment"] = df.apply(tag_segment, axis=1)

# [7] Regime
regime_info = _regime()

# [8] Peg/Wrapped/Stable-Maske
peg_mask = exclusion_mask(df, df.get("Kategorie", pd.Series()))

# [9] Scores
df = score_block(df, regime_info=regime_info)

# [10] Early Pass1
df_p1 = df.copy()
df_p1["breakout_score"] = np.nan
df_p1["vol_acc"] = 1.0
df_p1 = compute_early_score(df_p1, peg_mask=peg_mask, regime_info=regime_info)
df["early_prelim"] = df_p1["early_score"]

# [11] Kandidaten
def _pick_candidates(dfin, per_segment):
    out=[]
    for s, n in per_segment.items():
        sub = dfin[dfin["Segment"]==s].copy()
        if sub.empty: continue
        sub = sub.sort_values(["early_prelim","score_segment"], ascending=[False,False]).head(int(n))
        out.append(sub)
    return pd.concat(out, ignore_index=True) if out else dfin.head(0)
if LIGHT_BREAKOUT_ALL:
    cand_ids = df.loc[~df["mexc_pair"].isna(), "id"].astype(str).tolist()
else:
    cand_ids = _pick_candidates(df, CFG["BREAKOUT_PER_SEGMENT"])["id"].astype(str).tolist()

# [12] Breakout
t0 = time.perf_counter()
br = compute_breakout_for_ids(df, cand_ids, days=CFG["DAYS"], progress=True, light=bool(LIGHT_BREAKOUT_ALL))
if isinstance(br, pd.DataFrame) and not br.empty:
    df = df.merge(br, on="id", how="left")
else:
    for c in ["dist_90","dist_180","dist_365","p365","donch_width","vol_acc","vol_acc_7d","vol_acc_30d","z_break","z_donch","breakout_score","beta_btc","beta_eth","break_vol_mult","price_source"]:
        if c not in df.columns: df[c] = np.nan
df["price_source"] = df["price_source"].fillna("cg")
logging.info(f"[12] Breakout ok in {time.perf_counter()-t0:.2f}s")

# [13] Buzz
t0 = time.perf_counter()
df = add_buzz_metrics_for_candidates(
    df_in=df,
    top_n=min(CFG["BUZZ_TOPN"], len(df)),
    use_cp=USE_CRYPTOPANIC,
    mask_pegged=peg_mask,
    rss_news=None,
    cp_api_key=os.getenv("CRYPTOPANIC_API_KEY")
)
logging.info(f"[13] Buzz ok in {time.perf_counter()-t0:.2f}s")

# [14] Early final
t0 = time.perf_counter()
df = compute_early_score(df, peg_mask=peg_mask, regime_info=regime_info)
logging.info(f"[14] Early final ok in {time.perf_counter()-t0:.2f}s")

# [15] Rankings
def _keep_cols(dfin, extra=None):
    base = ["id","symbol","name","Segment","market_cap","total_volume","mexc_pair","score_global","score_segment","early_score"]
    ext = extra or []
    cols = [c for c in base+ext if c in dfin.columns]
    return dfin[cols].copy()
top25_global = df.sort_values("score_global", ascending=False).head(25)
top25_early  = df.sort_values("early_score", ascending=False).head(25)
seg_names = ["Hidden Gem","Emerging","Comeback","Momentum Gem","Balanced"]
top10_segments = {}
for s in seg_names:
    sub = df[df["Segment"]==s].copy()
    if not sub.empty: top10_segments[s] = sub.sort_values("early_score", ascending=False).head(10)
top10_all = pd.concat([v for v in top10_segments.values()], ignore_index=True) if top10_segments else df.head(0)

# [16] FullData (mit Buzz-Spalten)
full_data = make_fulldata(df)

# [17] Backtest
bt = pd.DataFrame()
if CFG.get("BACKTEST", False):
    bt = backtest_on_snapshot(df.sort_values("early_score", ascending=False), topk=20, days_list=[20,40,60], vs=VS)

# [18] Meta
from datetime import datetime as _dtu
meta = {
    "version": f"v14.5-{RUN_MODE}",
    "vs": VS,
    "min_volume_usd": str(int(MIN_VOLUME_USD)),
    "cap_min": str(CAP_MIN),
    "cap_max": str(CAP_MAX),
    "pages": str(CFG["PAGES"]),
    "breakout_per_segment": str(CFG["BREAKOUT_PER_SEGMENT"]),
    "buzz_topn": str(CFG["BUZZ_TOPN"]),
    "days": str(CFG["DAYS"]),
    "use_cryptopanic": str(USE_CRYPTOPANIC),
    "require_mexc": str(REQUIRE_MEXC),
    "light_breakout_all": "1" if LIGHT_BREAKOUT_ALL else "0",
    "allow_cg_fallback": os.getenv("ALLOW_CG_FALLBACK","0"),
    "buzz_half_life_h": os.getenv("BUZZ_HALF_LIFE_H","48"),
    "publisher_weights": os.getenv("BUZZ_PUBLISHER_WEIGHTS","{}"),
    "timestamp_utc": _dtu.now(timezone.utc).isoformat(),
}

# [19] Export
with pd.ExcelWriter(EXPORT_PATH, engine="xlsxwriter") as w:
    write_sheet(top25_global, "Top25_Global", w)
    for s, dseg in top10_segments.items():
        write_sheet(dseg, f"Top10_{s.replace(' ', '_')}", w)
    if not top10_all.empty:
        write_sheet(top10_all, "Top10_AllSegments", w)
    write_sheet(top25_early, "Top25_EarlySignals", w)
    write_sheet(full_data, "FullData", w)
    if CFG.get("BACKTEST", False) and not bt.empty:
        write_sheet(bt, "Backtest", w)
    write_meta_sheet(w, meta)

print("Export:", EXPORT_PATH)
try:
    display(top25_early.head(10)); display(full_data.head(10))
except Exception:
    pass


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Export: /content/drive/MyDrive/Colab results/Scanner_v14_5_output_standard_202510222126.xlsx


,id,symbol,name,market_cap,total_volume,price_change_percentage_7d_in_currency,price_change_percentage_30d_in_currency,ath_change_percentage,mexc_pair,Kategorie,tvl_usd,volume_mc_ratio,slope30,ath_drawdown_pct,circ_pct,Segment,mc_bucket,score_global,score_segment,early_prelim,breakout_score,z_break,z_donch,dist_90,dist_180,dist_365,p365,vol_acc,vol_acc_7d,vol_acc_30d,break_vol_mult,donch_width,beta_btc,beta_eth,price_source,buzz_48h,buzz_7d,buzz_level,buzz_acc,early_score,risk_regime,beta_pen
27,dash,DASH,Dash,543555262,138530793.0,-8.179776,109.466091,-97.08098,DASHUSDT,Cryptocurrency,NaN,0.254861,109.466091,-97.08098,NaN,Comeback,500m_2b,1.749456,1.749456,0.750000,0.793979,1.083922,0.117445,-18.364662,-18.364662,-32.978395,91.780822,0.930398,1.196814e+07,1.286346e+07,0.930398,165.204387,1.244891,0.812802,mexc,0.0,0.0,0.0,0.0,1.480679,risk_off,0.963266
22,beldex,BDX,Beldex,576481807,10875833.0,-2.527812,-11.510289,-82.88187,BDXUSDT,Unknown,NaN,0.018866,-11.510289,-82.88187,NaN,Balanced,500m_2b,0.207438,0.207438,0.113451,1.117619,1.171020,0.993014,-17.121668,-17.121668,-17.121668,64.109589,1.169234,8.370821e+05,7.159236e+05,1.169234,31.896796,0.127015,0.052029,mexc,0.0,0.0,0.0,0.0,1.456937,risk_off,1.000000
67,ribbita-by-virtuals,TIBBIR,Ribbita by Virtuals,321585667,4128980.0,80.322994,69.955508,-16.96271,TIBBIRUSDT,Unknown,NaN,0.012839,69.955508,-16.96271,NaN,Emerging,150m_500m,0.783974,0.783974,0.331937,1.452629,2.241046,-0.387009,-1.851288,-1.851288,NaN,99.056604,1.482243,3.698754e+05,2.495376e+05,1.482243,299.440525,-0.347437,-0.200950,mexc,0.0,0.0,0.0,0.0,1.395142,risk_off,1.000000
86,undeads-games,UDS,Undeads Games,253600365,1113050.0,7.431844,53.222960,-10.35767,UDSUSDT,Unknown,NaN,0.004389,53.222960,-10.35767,NaN,Emerging,150m_500m,0.003335,0.003335,0.254157,1.824280,2.370769,0.549139,0.000000,0.000000,0.000000,100.000000,1.415876,5.706551e+05,4.030404e+05,1.415876,158.650628,-0.418320,-0.090696,mexc,0.0,0.0,0.0,0.0,1.390051,risk_off,1.000000
164,debridge,DBR,deBridge,123914436,24993934.0,6.098500,49.667231,-36.33253,DBRUSDT,Unknown,NaN,0.201703,49.667231,-36.33253,NaN,Hidden Gem,le150m,0.306869,0.306869,0.352194,1.960296,2.331348,1.094510,-0.562588,-0.562588,-34.922680,94.520548,1.214989,8.086562e+04,6.655667e+04,1.214989,71.160327,0.718254,0.491454,mexc,0.0,0.0,0.0,0.0,1.277859,risk_off,1.000000
176,edu-coin,EDU,Open Campus,110991843,39162718.0,11.861366,27.879932,-88.83985,EDUUSDT,Unknown,NaN,0.352843,27.879932,-88.83985,NaN,Comeback,le150m,0.577746,0.577746,0.232363,1.935131,2.370769,0.918641,0.000000,-0.638298,-75.274653,63.013699,1.362909,3.726394e+06,2.734146e+06,1.362909,100.644468,1.453459,0.958815,mexc,0.0,0.0,0.0,0.0,1.144974,risk_off,0.931981
287,auction,AUCTION,Bounce,55029719,76820335.0,20.906908,-12.024256,-88.98633,AUCTIONUSDT,Unknown,NaN,1.395979,-12.024256,-88.98633,NaN,Hidden Gem,le150m,0.957358,0.957358,0.012890,0.048013,-0.000163,0.160423,-33.835675,-44.941193,-84.949839,3.287671,2.887878,7.556264e+06,2.616545e+06,2.887878,106.887441,1.070778,0.723306,mexc,0.0,0.0,0.0,0.0,0.974848,risk_off,0.989383
221,xpin-network,XPIN,XPIN Network,79153310,92355702.0,336.194470,360.038201,-6.09587,XPINUSDT,Unknown,NaN,1.166795,360.038201,-6.09587,NaN,Momentum Gem,le150m,2.400000,2.400000,0.375000,0.523638,2.370769,-3.786335,0.000000,NaN,NaN,100.000000,2.818197,3.298245e+06,1.170339e+06,2.818197,839.253704,-1.222788,-1.097820,mexc,0.0,0.0,0.0,0.0,0.946261,risk_off,1.000000
1,morpho,MORPHO,Morpho,964516867,32177965.0,-4.834003,0.120844,-55.81979,MORPHOUSDT,Unknown,NaN,0.033362,0.120844,-55.81979,NaN,Balanced,500m_2b,0.124506,0.124506,0.234529,0.205950,0.174418,0.279527,-31.344230,-31.344230,-53.071272,52.976190,1.007478,1.902416e+06,1.888296e+06,1.007478,96.494830,0.506277,0.149591,mexc,0.0,0.0,0.0,0.0,0.875982,risk_off,1.000000
80,basic-attention-token,BAT,Basic Attention,265265979,26522449.0,-6.451404,22.875187,-90.66954,BATUSDT,Unknown,NaN,0.099984,22.875187,-90.66954,NaN,Comeback,150m_500m,0.150007,0.150007,0.1

,id,symbol,name,Kategorie,Segment,market_cap,total_volume,mexc_pair,price_source,tvl_usd,dist_90,dist_180,dist_365,p365,donch_width,vol_acc,vol_acc_7d,vol_acc_30d,z_break,z_donch,breakout_score,break_vol_mult,beta_btc,beta_eth,buzz_48h,buzz_7d,buzz_acc,score_global,score_segment,early_score,early_prelim,risk_regime,beta_pen
0,immutable-x,IMX,Immutable,Unknown,Balanced,993148476,38061080.0,IMXUSDT,mexc,NaN,-40.016444,-40.016444,-75.989657,18.630137,78.266332,0.609187,2.169736e+06,3.561693e+06,-0.433261,0.211502,-0.239832,0.609187,1.713733,1.145592,0.0,0.0,0.0,0.036628,0.036628,-0.255262,-0.043661,risk_off,0.892940
1,morpho,MORPHO,Morpho,Unknown,Balanced,964516867,32177965.0,MORPHOUSDT,mexc,NaN,-31.344230,-31.344230,-53.071272,52.976190,96.494830,1.007478,1.902416e+06,1.888296e+06,0.174418,0.279527,0.205950,1.007478,0.506277,0.149591,0.0,0.0,0.0,0.124506,0.124506,0.875982,0.234529,risk_off,1.000000
2,pancakeswap-token,CAKE,PancakeSwap,Unknown,Balanced,936171570,189474829.0,CAKEUSDT,mexc,NaN,-37.408425,-37.408425,-37.408425,80.821918,86.427657,0.607593,8.193201e+06,1.348469e+07,-0.250512,0.214891,-0.110891,0.607593,1.401856,0.989875,0.0,0.0,0.0,0.591991,0.591991,0.082459,0.272092,risk_off,0.939722
3,spx6900,SPX,SPX6900,Ethereum (ETH) Token (ERC-20),Balanced,886423823,33828564.0,NAN,cg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,-0.522808,-0.522808,0.094841,0.112519,risk_off,1.000000
4,fasttoken,FTN,Fasttoken,Ethereum (ETH) Token (ERC-20),Balanced,872885087,21357584.0,FTNUSDT,mexc,NaN,-59.425417,-59.425417,-59.425417,4.931507,157.536537,0.699201,1.142521e+06,1.634038e+06,-1.793286,-0.705574,-1.466972,0.699201,0.084068,0.033223,0.0,0.0,0.0,-0.447590,-0.447590,-1.007084,-0.340909,risk_off,1.000000
5,doublezero,2Z,DoubleZero,Solana (SOL) Token,Balanced,825315826,81028208.0,2ZUSDT,cg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,0.000000,-0.017678,0.000000,risk_off,1.000000
6,injective-protocol,INJ,Injective,Exchange,Balanced,819611121,94859098.0,NAN,cg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,-0.061846,-0.061846,-0.115546,-0.097868,risk_off,1.000000
7,celestia,TIA,Celestia,Osmosis Ecosystem,Balanced,808064550,86327764.0,NAN,cg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,-0.134558,-0.134558,-0.140440,-0.122762,risk_off,1.000000
8,lido-dao,LDO,Lido DAO,Ethereum (ETH) Token (ERC-20),Balanced,790148646,121993394.0,LDOUSDT,mexc,NaN,-42.953499,-42.953499,-63.010934,25.205479,90.994674,0.666469,1.378769e+06,2.068767e+06,-0.639066,0.068058,-0.426929,0.666469,1.838445,1.350685,0.0,0.0,0.0,0.285957,0.285957,-0.189158,0.016735,risk_off,0.874233
9,blockstack,STX,Stacks,Platform,Balanced,766982225,21271803.0,STXUSDT,mexc,NaN,-50.697674,-59.386973,-84.576210,1.095890,103.791469,0.988108,8.193307e+05,8.291916e+05,-1.181716,-0.177895,-0.880569,0.988108,1.751216,1.004392,0.0,0.0,0.0,-0.171640,-0.171640,0.021484,-0.074136,risk_off,0.887318
